In [1]:
import pandas as pd
data  =  pd.read_csv('/Users/adham/Desktop/EMO_REC/Iemocap_clean_data.csv')
data.head()

,Unnamed: 0,session,method,gender,emotion,n_annotators,agreement,path,transcript,duration
0,0,1,script,F,neu,3,3,Session1/sentences/wav/Ses01F_script02_1/Ses01...,Fine.\n,2.070000
1,1,1,script,F,fru,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,[BREATHING]\n,1.502438
2,2,1,script,F,neu,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,That's not your flashlight.\n,2.180000
3,3,1,script,F,ang,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,You keep saying my flashlight like it's just ...,2.970000
4,4,1,script,F,ang,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...,How's that supposed to make me feel?\n,1.819937


In [2]:
import pandas as pd
import numpy as np
data_text = pd.DataFrame()
data_text['transcript'] = data['transcript']
data_text['emotion'] = data['emotion']

In [3]:
data_text.head()

,transcript,emotion
0,Fine.\n,neu
1,[BREATHING]\n,fru
2,That's not your flashlight.\n,neu
3,You keep saying my flashlight like it's just ...,ang
4,How's that supposed to make me feel?\n,ang


lowercassing -
stop word removal

In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
data_text['transcript'] = data_text['transcript'].str.lower()

# Define a function to remove stop words from a single string
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Apply the 'remove_stopwords' function to the 'Text' column
data_text['transcript'] = data_text['transcript'].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to /Users/adham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


stemming


In [5]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
# Define a function to perform stemming on a single string
def perform_stemming(text):
    stemmer = PorterStemmer()
    tokens = word_tokenize(text)
    stemmed_words = []
    for token in tokens:
        stemmed_words.append(stemmer.stem(token))
    return ' '.join(stemmed_words)

# Apply the 'perform_stemming' function to the 'Text' column
data_text['transcript'] = data_text['transcript'].apply(perform_stemming)

# Print the updated DataFrame
#print(data_text)

[nltk_data] Downloading package punkt to /Users/adham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adham/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/adham/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


remove special characters

In [6]:
import pandas as pd
import re


# Define a function to remove non-alphabetic characters from a single string
def remove_non_alphabetic(text):
    pattern = r'[^a-zA-Z\s]'
    return re.sub(pattern, '', text)

# Apply the 'remove_non_alphabetic' function to the 'Text' column
data_text['transcript'] = data_text['transcript'].apply(remove_non_alphabetic)



In [7]:
data_text['transcript']

0                                                   fine 
1                                                 breath 
2                                      that s flashlight 
3                          keep say flashlight like your 
4                                 how s suppos make feel 
                              ...                        
6780                                                yeah 
6781    know what  go pass  eventu  mean it ll never p...
6782                              that s best way put it 
6783                                             absolut 
6784    know what  die  i d realli want cri realli ups...
Name: transcript, Length: 6785, dtype: object

In [8]:
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer

# Create a sample DataFrame with a column of preprocessed text data
#df = pd.DataFrame({'Text': ['sample text', 'another text']})

# Load the pre-trained BERT model
model_name = 'bert-base-uncased'
model = AutoModel.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to generate embeddings for a single string
def generate_embedding(text):
    # Tokenize the text data
    tokens = tokenizer.encode(text, add_special_tokens=True)
    # Convert the tokens to a PyTorch tensor
    tokens_tensor = torch.tensor(tokens).unsqueeze(0)
    # Generate the embeddings for the input tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        embedding = outputs.last_hidden_state[:, 0, :]
    return embedding.numpy()[0]

# Apply the 'generate_embedding' function to the 'Text' column
data_text['Embeddings'] = data_text['transcript'].apply(generate_embedding)

# Print the updated DataFrame
#print(df)


/opt/anaconda3/envs/emo_rec/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly 

In [12]:
data_text['Embeddings'][0].shape

(768,)

In [ ]:
data_text.to_csv("IemoCap_embeddings_text.csv")